In [5]:
import os
import sys

# Go two levels up from /home/ubuntu/iDynamics/Evaluations/Policy1_eval_Graph_dynamics
root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

# Now 'Evaluations' is visible as a top-level package
from Evaluations.policyExtender.my_cluster_utils import (
    gather_all_nodes,
    build_nodeinfo_objects,
    gather_all_pods,
    build_podinfo_objects
)

# etc.


ModuleNotFoundError: No module named 'my_policy_interface'

In [ ]:
request_A_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/compose-post.lua"
request_B_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/read-home-timeline.lua"
request_C_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/read-user-timeline.lua"

request_mix_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/mixed-workload.lua" # default mix percentage: 60% A, 30% B, 10% C

script_path = [
    request_A_script_path,
    request_B_script_path,
    request_C_script_path,
    request_mix_script_path
]

url = [
    #diffferent namespace of social-network Application
    "http://nginx-thrift.social-network.svc.cluster.local:8080"       # in namespace of "social-network"
#     "http://nginx-thrift.social-network2.svc.cluster.local:8080",
#    "http://nginx-thrift.social-network3.svc.cluster.local:8080"
]

duration = "2m" # eg., "30s", "1m", 5m", "10m", "15m"


for i in range(0, len(script_path)):
    Policy1_demo_workloads(script_path[i], url[0], request_inerval = duration)
        # 1) Gather cluster data
    raw_nodes = gather_all_nodes()
    candidate_nodes = build_nodeinfo_objects(raw_nodes)

    # 1) Gather cluster data
    raw_nodes = gather_all_nodes()
    candidate_nodes = build_nodeinfo_objects(raw_nodes)

    # Suppose your social-network microservices are in namespace "social-network"
    raw_pods = gather_all_pods(namespace="social-network")
    pods = build_podinfo_objects(raw_pods)

    # 2) (Optional) Build a call graph from 'graph_builder.py'
    #    This might be used by Policy1 to place heavily communicating microservices.
    G = build_call_graph(namespace="social-network")
    # Convert the NetworkX graph to a simple dict: (src, dst) -> traffic_kb
    traffic_matrix = {}
    for (u, v, data) in G.edges(data=True):
        traffic_matrix[(u, v)] = data.get("weight", 0.0)  # store as KB

    # 3) Initialize each policy with scenario-specific config
    policy1 = Policy1CallGraphAware()
    policy1.initialize_policy(config={"traffic_matrix": traffic_matrix})

    # policy2 = Policy2LatencyAware()
    # policy2.initialize_policy(config={"latency_threshold": 5.0})  # example

    # policy3 = Policy3BandwidthAware()
    # policy3.initialize_policy(config={"high_traffic_threshold": 300.0})  # example

    # 4) Choose which scenario you want to test
    # Scenario 1: call-graph dynamics => we expect policy1 to shine
    print("=== Scenario1: Call-Graph_Aware ===")
    decisions_s1 = policy1.schedule_all(pods, candidate_nodes)
    for dec in decisions_s1:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

    # # Scenario 2: cross-node delays => policy2 might do better
    # print("\n=== Scenario2: Latency_Aware ===")
    # decisions_s2 = policy2.schedule_all(pods, candidate_nodes)
    # for dec in decisions_s2:
    #     print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

    # # Scenario 3: cross-node bandwidth changes => policy3 might do better
    # print("\n=== Scenario3: Bandwidth_Aware ===")
    # decisions_s3 = policy3.schedule_all(pods, candidate_nodes)
    # for dec in decisions_s3:
    #     print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

